In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from deeplkt.datasets.dataset import *
from torch.utils.data import DataLoader
from deeplkt.models.pure_lkt import PureLKTNet
from deeplkt.models.lkt_alexsobel import LKTAlexSobelNet

from deeplkt.models.lkt_vggsobel import LKTVGGSobelNet
from deeplkt.models.lkt_vggimproved import LKTVGGImproved
from deeplkt.models.base_model import BaseModel

from deeplkt.utils.util import dotdict
from deeplkt.tracker.lkt_tracker import LKTTracker
from deeplkt.config import *

/home/sudeep/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudeep/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudeep/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sudeep/anaconda3/envs/pytorch/lib/pyt

In [3]:

use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda") if use_cuda else torch.device("cpu")



True


In [4]:
#!/usr/bin/env python
# coding: utf-8


# from pytorch_practise import *


vot_root_dir = '../../data/VOT/'
alov_root_dir = '../../data/ALOV/'


vot = VotDataset(os.path.join(vot_root_dir,
                       'VOT_images/'),
                 os.path.join(vot_root_dir,
                       'VOT_ann/'),
                 os.path.join(vot_root_dir,
                       'VOT_results/'), 
                 device)

alov = AlovDataset(os.path.join(alov_root_dir,
                       'ALOV_images/'),
                   os.path.join(alov_root_dir,
                       'ALOV_ann/'),
                   os.path.join(alov_root_dir,
                       'ALOV_results/'), 
                       device)


# train_loader = DataLoader(alov, batch_size=1, shuffle=False)



Parsing VOT dataset...
VOT dataset parsing done.
Total number of annotations in VOT dataset = 10651
Total number of videos in VOT dataset = 26
Parsing ALOV dataset...
ALOV dataset parsing done.
Total number of videos in ALOV dataset = 307
Total number of annotations in ALOV dataset = 15877


In [6]:
import cv2
img = vot.show_image_with_gt(0,1)
cv2.imwrite("vot_0_1.jpeg", img)

True

In [17]:
from deeplkt.config import *
params = dotdict({
    'mode' : MODE,
    'max_iterations' : MAX_LK_ITERATIONS,
    'epsilon' : EPSILON,
    'info': "VGGSobel LKT"
})
# lr = 0.0005
# momentum = 0.5


net = LKTVGGImproved(device, params)
tracker = LKTTracker(net)
train_params = dotdict({
    'batch_size' : BATCH_SIZE,
    'val_split' : VALIDATION_SPLIT,
    'train_examples':TRAIN_EXAMPLES,
    'shuffle_train': SHUFFLE_TRAIN,
    'random_seed': RANDOM_SEED,
    'lr': LR,
    'momentum': MOMENTUM,
    'l2': L2

})

model = BaseModel(tracker, 'checkpoint', 'logs', train_params)



In [7]:
for p in net.parameters():
    if(p.requires_grad):
        print(p.shape)


torch.Size([1, 3000, 1, 3, 3])
torch.Size([1, 3000, 1, 3, 3])


In [8]:
# print(net)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(net)


54000

In [30]:
model.load_checkpoint(45)
model.eval_model(vot, 1)


checkpoint/VGGSobel LKT/45-checkpoint.pth
Evaluating dataset for video  1
Time taken =  87.22712922096252
Mean iou =  0.014401258432961176


0.014401258432961176

In [26]:
mp = model.nn.model.state_dict()
for i, k in enumerate(mp):
    if(i < 4 and i %2 == 0):
        print(mp[k])

tensor([[[[[-9.9393e-01, -3.6622e-02,  9.2164e-01],
           [-1.9538e+00, -3.7756e-02,  1.8793e+00],
           [-9.9468e-01, -3.7436e-02,  9.2091e-01]]],


         [[[-1.0711e+00, -1.1318e-01,  8.4654e-01],
           [-2.0285e+00, -1.1169e-01,  1.8068e+00],
           [-1.0677e+00, -1.0959e-01,  8.5013e-01]]],


         [[[-9.4848e-01,  9.3161e-03,  9.6728e-01],
           [-1.9078e+00,  8.7496e-03,  1.9255e+00],
           [-9.5028e-01,  7.5542e-03,  9.6546e-01]]],


         ...,


         [[[-9.5783e-01,  8.8215e-04,  9.5971e-01],
           [-1.9166e+00,  8.0595e-04,  1.9184e+00],
           [-9.5787e-01,  8.4408e-04,  9.5967e-01]]],


         [[[-9.6409e-01, -5.4916e-03,  9.5332e-01],
           [-1.9228e+00, -5.4966e-03,  1.9121e+00],
           [-9.6431e-01, -5.7100e-03,  9.5311e-01]]],


         [[[-9.5881e-01, -2.0641e-05,  9.5877e-01],
           [-1.9175e+00,  5.2860e-06,  1.9175e+00],
           [-9.5881e-01, -1.4590e-05,  9.5877e-01]]]]],
       device='cuda:0', 

In [8]:

model.load_checkpoint(74)


LKTVGGSobelNet(
  (pad): ReflectionPad2d((1, 1, 1, 1))
  (vgg): VGGImproved(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (vgg): VGG(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): 

In [11]:
from time import time
start_t = time()
model.eval_model(vot, 2)
end_t = time()
print(end_t - start_t)

Evaluating dataset for video  2
(1, 8)
(1, 240, 320, 3)
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3, 127, 127])
torch.Size([1, 3